In [1]:
import numpy as np
import ng_link
from ng_link import NgState, link_utils

import json
from pathlib import Path
from typing import Union
import boto3
from botocore.exceptions import ClientError

PathLike = Union[Path, str]

In [2]:
def _zyx_vector_to_5x6(zyx_vector: np.ndarray):
    output = np.zeros(5, 6)
    output[2, 5] = zyx_vector[0]
    output[3, 5] = zyx_vector[1]
    output[4, 5] = zyx_vector[0]
    return output

def read_json(bucket_name: str, json_key: PathLike) -> dict:
    """
    Reads a json content hosted in S3

    Parameters
    ---------------
    bucket_name: str
        Bucket name

    json_key: PathLike
        Path where the json is stored in S3

    Returns
    ---------------
    dict
        Dictionary with the json content
    """
    s3 = boto3.resource("s3")
    content_object = s3.Object(bucket_name, json_key)

    try:
        file_content = content_object.get()["Body"].read().decode("utf-8")
        json_content = json.loads(file_content)
    except ClientError as ex:
        if ex.response["Error"]["Code"] == "NoSuchKey":
            json_content = {}
            print(
                f"An error occurred when trying to read json file from {json_key}"
            )
        else:
            raise

    return json_content

# Simply applies the same registration across channels, b/c coreg is extra effort for little gain
# Notice the same tile layout, cx, and cy will be applied to each of the nested lists of paths. 
def create_ng_link(tile_paths: list[list[str]], 
                   tile_layout: np.ndarray, 
                   cx: np.ndarray, 
                   cy: np.ndarray,
                   subtract_offsets: False, 
                   max_dr: int = 200,
                   opacity: float = 1.0,
                   blend: str = "default",
                   output_json_path: str = ".") -> None:
    
    # Pre-processing
    cx[np.isnan(cx)] = 0    
    cy[np.isnan(cy)] = 0    

    # Determine tile translations
    tile_translations_zyx = np.zeros((tile_layout.shape[0], tile_layout.shape[1], 3))
    for xi in range(tile_layout.shape[0]):
        for yi in range(tile_layout.shape[1]):
            if xi == 0 and yi == 0:  # TL corner tile is fixed
                continue
            
            if xi == 0: # Tile on top border 
                h_shift = cx[:, 0, xi, yi]
                tile_translations_zyx[xi, yi] = h_shift + tile_translations_zyx[xi - 1, yi]

            elif yi == 0: # Tile on left border
                v_shift = cy[:, 0, xi, yi]
                tile_translations_zyx[xi, yi] = v_shift + tile_translations_zyx[xi, yi - 1]

            else:
                h_shift = cx[:, 0, xi, yi]
                v_shift = cy[:, 0, xi, yi]
                tile_translations_zyx[xi, yi] = v_shift + tile_translations_zyx[xi - 1, yi - 1]

    print('Net Coarse Registration')
    print(tile_translations_zyx)

    # Subtracts initial positioning of tiles specified in .zattrs file (exaspim datasets)
    if subtract_offsets:         
        for xi in range(tile_layout.shape[0]):
            for yi in range(tile_layout.shape[1]):
                tile_id = tile_layout[xi, yi]

                print('tile_id', tile_id)
                t_path = tile_paths[0][tile_id]

                json_file = read_json('aind-open-data', f'{t_path}/.zattrs')

                print('json_file', json_file)

                offset = np.array(json_file["multiscales"][0]["datasets"][0]["coordinateTransformations"][1]["translation"])
                offset_zyx = offset[2:5] / np.array([4, 3, 3])  # FIXME -- adjust scale accordingly

                tile_translations_zyx[xi, yi] = tile_translations_zyx[xi, yi] - offset_zyx


        print('Net Coarse Registration, Subtract Offsets')
        print(tile_translations_zyx)

    # Determine voxel size
    vox_sizes = (2048, 2660, 512) # FIXME: Hardcoded for now

    # Determine color
    channel: int = link_utils.extract_channel_from_tile_path(tile_paths[0][0])
    hex_val: int = link_utils.wavelength_to_hex(channel)
    hex_str = f"#{str(hex(hex_val))[2:]}"
    
    # Generate input config
    layers = []  # Nueroglancer Tabs
    input_config = {
        "dimensions": {
            "x": {"voxel_size": vox_sizes[0], "unit": "microns"},
            "y": {"voxel_size": vox_sizes[1], "unit": "microns"},
            "z": {"voxel_size": vox_sizes[2], "unit": "microns"},
            "c'": {"voxel_size": 1, "unit": ""},
            "t": {"voxel_size": 0.001, "unit": "seconds"},
        },
        "layers": layers,
        "showScaleBar": False,
        "showAxisLines": False,
    }

    for channel_tile_paths in tile_paths:
        sources = []  # Tiles within tabs
        layers.append(
            {
                "type": "image",  # Optional
                "source": sources,
                "channel": 0,  # Optional
                "shaderControls": {
                    "normalized": {"range": [0, max_dr]}
                },  # Optional  # Exaspim has low HDR
                "shader": {
                    "color": hex_str,
                    "emitter": "RGB",
                    "vec": "vec3",
                },
                "visible": True,  # Optional
                "opacity": opacity,
                "name": f"CH_{channel}",
                "blend": blend,
            }
        )

        for tile_id, tr_zyx in zip(tile_layout, tile_translations_zyx):
            url = f"s3://aind-open-data/{channel_tile_paths[tile_id]}"
            sources.append(
                {"url": url, "transform_matrix": _zyx_vector_to_5x6(tr_zyx)}
            )
    
    # Generate the link
    neuroglancer_link = NgState(
        input_config=input_config,
        mount_service="s3",
        bucket_path="aind-open-data",
        output_json=output_json_path,
    )
    neuroglancer_link.save_state_as_json()
    print(neuroglancer_link.get_url_link())

    return input_config

FULL TESTING SCRIPT

In [5]:
# Everything required for registration is here, 
# cannot wrap into a function yet due to tile layout, inferred from neuroglancer. 

import numpy as np
import zarr_io
import coarse_registration

# Data Prep
bucket = 'aind-open-data'
# dataset = 'exaSPIM_615296_2022-09-28_11-47-06/exaSPIM/'
dataset = 'exaSPIM_H17.24.006-CX55-31B_2023-05-11_14-59-09/exaSPIM.zarr/'
downsampling_exp = 2

# Exapsim dataset
# tile_volumes = []  # Filled with x in outer loop and y in inner loop
# tile_paths = []
# for x in range(5):
#     for y in range(3):
#         path = f'tile_x_000{x}_y_000{y}_z_0000_ch_488/{downsampling_exp}'  # Modified
#         tile = zarr_io.open_zarr_s3(bucket, dataset + path)
#         tile_volumes.append(tile.T[:,:,:,0,0])
#         tile_paths.append(dataset + path)
# tile_layout = np.array([[14, 11, 8, 5, 2], 
#                         [13, 10, 7, 4, 1], 
#                         [12,  9, 6, 3, 0]])


# Few outliers, let's tweak the global variables

# Axonal dataset
tile_layout = np.array([[24, 20, 16, 12, 8, 4, 0],
                        [25, 21, 17, 13, 9, 5, 1],
                        [26, 22, 18, 14, 10, 6, 2],
                        [27, 23, 19, 15, 11, 7, 3]])
tile_volumes = []
tile_paths = []
for x in range(7):
    for y in range(4):
        path = f'tile_x_000{x}_y_000{y}_z_0000_ch_488.zarr'
        tile = zarr_io.open_zarr_s3(bucket, dataset + path + f'/{downsampling_exp}')
        tile_volumes.append(tile.T[:,:,:,0,0])
        tile_paths.append(dataset + path)


# Entirety of the registration
cx, cy = coarse_registration.compute_coarse_offsets(tile_layout, tile_volumes)

# Entirety of the viewing 
create_ng_link([tile_paths], tile_layout, cx, cy, True)

Left Id: 24, Right Id: 20
Left: (0, 0), Right: (0, 1) [361.  -1. -50.]
Left Id: 20, Right Id: 16
Left: (0, 1), Right: (0, 2) [395.  -1.   4.]
Left Id: 16, Right Id: 12
Left: (0, 2), Right: (0, 3) [361.  -1. -21.]
Left Id: 12, Right Id: 8
Left: (0, 3), Right: (0, 4) [361.  -1. -50.]
Left Id: 8, Right Id: 4
Left: (0, 4), Right: (0, 5) [361.  -1. -50.]
Left Id: 4, Right Id: 0
Left: (0, 5), Right: (0, 6) [361.   0. -48.]
Left Id: 25, Right Id: 21
Left: (1, 0), Right: (1, 1) [361.  -1. -50.]
Left Id: 21, Right Id: 17
Left: (1, 1), Right: (1, 2) [363.  -1. -46.]
Left Id: 17, Right Id: 13
Left: (1, 2), Right: (1, 3) [363.  -1. -50.]
Left Id: 13, Right Id: 9
Left: (1, 3), Right: (1, 4) [361.  -1. -50.]
Left Id: 9, Right Id: 5
Left: (1, 4), Right: (1, 5) [359.  -1. -50.]
Left Id: 5, Right Id: 1
Left: (1, 5), Right: (1, 6) [361.  -1. -50.]
Left Id: 26, Right Id: 22
Left: (2, 0), Right: (2, 1) [366.  -1. -50.]
Left Id: 22, Right Id: 18
Left: (2, 1), Right: (2, 2) [364.  -1. -49.]
Left Id: 18, Rig

AttributeError: 'NoneType' object has no attribute 'group'

In [20]:
t_path = 'exaSPIM_H17.24.006-CX55-31B_2023-05-11_14-59-09/exaSPIM.zarr/tile_x_0000_y_0000_z_0000_ch_488.zarr'

json_file = read_json('aind-open-data', f'{t_path}/.zattrs')
offset = np.array(json_file["multiscales"][0]["datasets"][0]["coordinateTransformations"][1]["translation"])
offset_zyx = offset[2:5]

offset_zyx / np.array([4, 3, 3])

# json_file["multiscales"][0]["datasets"][0]["coordinateTransformations"][1]["translation"]



array([ -512.        , -8091.36523438, -1021.26936849])

In [ ]:
def create_raw_ng_link(tile_paths: list[list[str]], 
                       tile_layout: np.ndarray,
                       max_dr: int = 200,
                       opacity: float = 1.0,
                       blend: str = "default",
                       output_json_path: str = "."):
    cx = cy = np.zeros((3, 1, tile_layout.shape[0], tile_layout.shape[1]))
    create_ng_link(tile_paths, 
                   tile_layout,
                   cx, 
                   cy,
                   max_dr,
                   opacity,
                   blend,
                   output_json_path)

In [ ]:
# Tweaks: 
# - (DONE) Tweak patches for accurate phase correlation 
# - (DONE) Adjust offset scale for 'subtract offsets'
# - (DONE) Set nan -> zero when summing two coarse offsets


# - Set inital offset for top left corner tile
# (Believe the learned offsets are corect. All offsets are defined in terms of top left corner as origin.)
# - Do not add the z component
# - See if this is correct, curious if we need to use the coarse mesh output.



# - Outlier detection-- essentially ignore bad phase correlation results
# and keep the default stage coordinates. 
# Hueristic: 
# - Check if there is actually values in the search space
#   If there is not enough stuff, do not do anything? 
#   Probably a preprocessing step, simply define a mask and reference mask.

# Expected Result-- Register/Wiggle the interior tiles, and 
# tiles that do not have enough content to be related to the neighbor tile
# is not wiggled -- in theory, no signal = background.
# Unless phase correlation fails in the center of the image-- that's an issue. 

# For those cases, can develop a seperate algorithm. 


In [ ]:
# What's left: 
# - Testing script
# - Move the code into repo generated by aind-capsule-template
# - JAX capsule environment
# - Capsule Inputs (not technically necessary right now)

# (That should be enough work for tomorrow)